In [ ]:
import importlib
import sys

import captum
import torch
import pytorch_lightning
from captum.attr import NoiseTunnel, Saliency
import numpy as np
from tqdm import tqdm
from rich.progress import track
import seaborn as sns
import optuna
import utils
importlib.reload(utils)
importlib.reload(sys.modules['utils'])
import saliency
importlib.reload(saliency)
importlib.reload(sys.modules['saliency'])
from saliency import get_all_attrs, calc_median


sys.path.append("../artemis")
from src.models import LitBlockLSTM, LitLSTM
import os
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
def get_attrs(model, absolute=False, block_size=24, target_idx=None, desired_target_val=None):
    # init
    if target_idx is None:
        target_idx = 0
    model.batch_size = 1
    dl = model.val_dataloader()
    epochs = 1
    model.cuda()
    sal_model = saliency.get_sal_model(model, idx=target_idx)
    # collect
    attrs = []
    for epoch in tqdm(range(epochs), disable=epochs == 1):
        for data, target, idcs, lens in tqdm(dl, disable=epochs != 1): 
            # Cut padded values out:
            data = data[0, :lens[0]].unsqueeze(0)
            # To GPU:
            data = data.cuda()
            # Add requires grad for saliency:
            data.requires_grad = True
            if len(data[0]) < block_size:
                continue
            split_target = torch.split(target, block_size, dim=1)
            for split_idx, _pat_split in enumerate(torch.split(data, block_size, dim=1)):
                if split_idx == (len(data[0]) // block_size) - 1 and len(data[0]) % block_size != 0:
                    break
                #print(split_target[split_idx][0, -1, target_idx])
                if desired_target_val is not None and split_target[split_idx][0, -1, target_idx] != desired_target_val:
                    #print("SKIP")
                    continue
                # Calculate Atrribution:
                out = model(_pat_split)
                attribution = sal_model.attribute(_pat_split, nt_type="smoothgrad", n_samples=50,
                                                  stdevs=0.01, abs=absolute, target=0).cpu()
                attrs.append(attribution.squeeze(0))
    #attrs = pad_and_mask(attrs)
    attrs = torch.stack(attrs).numpy()
    return attrs

In [ ]:
import matplotlib.pyplot as plt
def plot_mean_feature_attribution(mean_feature_attr, feature_names, title, folder=None):
    # Plot:
    plt.figure(figsize=(20, 30))
    plt.barh(range(len(mean_feature_attr)), mean_feature_attr, tick_label=feature_names)
    plt.rcParams.update({'font.size': 10})
    plt.yticks(fontsize=12)
    plt.title(title)
    if folder is None:
        folder = ""
    else:
        os.makedirs(folder, exist_ok=True)
    plt.savefig(folder + title + ".pdf")
    plt.close()

In [ ]:
def normalize_and_sort_mean_attribution(attrs, feature_names, feature_weights=None):
    if torch.is_tensor(feature_names):
        feature_names = feature_names.tolist()
    attr = attrs.sum(0).sum(0)
    print("Normed attr shape: ", attr.shape)
    # Apply weighting:
    if feature_weights is not None:
        mean_feature_attr *= feature_weights
    # Normalize it:
    attr /= np.abs(attr).max()
    attr = attr.squeeze()
    # Sort them:
    zipped_sort = sorted(zip(attr, feature_names), key=lambda x: x[0], reverse=True)
    sorted_attr, sorted_names = zip(*zipped_sort)
    sorted_attr = np.stack(sorted_attr)
    # Sort them also for abs value for saliency evaluation:
    zipped_sort = sorted(zip(np.abs(attr), feature_names), key=lambda x: x[0], reverse=True)
    _, sorted_names_abs_vals = zip(*zipped_sort)
    return sorted_attr, sorted_names, sorted_names_abs_vals

In [ ]:
def plot_saliency_map(attribution, feature_names, path_title, all_positive=False, folder=None, outcome=None, pred_outcome=None):
    if all_positive:
        cmap = None
        vmin = 0
        vmax = 1
    else:
        # Palette from orange to blue with maximum contrast
        cmap = sns.diverging_palette(15, 240, s=99, l=50, as_cmap=True)
        #cmap = "coolwarm"
        vmin = -1
        vmax = 1
    # Normalize:
    attribution = attribution / np.max(np.abs(attribution))
    # Plot:
    plt.figure(figsize=(75, 25), dpi=300)
    plt.rcParams.update({'font.size': 20})
    ax = sns.heatmap(attribution, yticklabels=feature_names, cbar=True, cmap=cmap,
                    vmin=vmin, vmax=vmax)
    ax.set(xlabel='Time step', ylabel='Feature')
    plot_title = "Feature Attribution Heatmap for Outcome Prediction"
    if outcome is not None:
        plot_title += " - Outcome: " + str(int(outcome)) + " - Pred Outcome: " + str(round(pred_outcome, 3))
    plt.title(plot_title)
    if folder is None:
        folder = ""
    else:
        os.makedirs(folder, exist_ok=True)
    plt.savefig(folder + path_title + ".jpg")
    plt.close()

In [ ]:
import numpy as np

def create_plots(attrs, name, feature_names, absolute=False):
    plot_folder = name + '/'
    os.makedirs(plot_folder, exist_ok=True)
    # store attrs:
    print("attr shape: ", attrs.shape)
    attrs.dump(f'{plot_folder}attrs.npy')
    # feature attribution
    sorted_attr, sorted_names, sorted_names_abs_vals = normalize_and_sort_mean_attribution(attrs, feature_names)
    plot_mean_feature_attribution(sorted_attr, sorted_names, f'Saliency', folder=plot_folder)
    # heatmap
    mean_map = np.moveaxis(np.sum(attrs, axis=0), 1, 0)
    title = f'saliency_heatmap'
    plot_saliency_map(mean_map, feature_names, title, all_positive=absolute, folder=plot_folder)
    # Importance over time
    over_time = np.abs(attrs).sum(0).mean(-1)
    over_time /= over_time.sum()
    title = f'importance_over_time'
    plt.figure(figsize=(7, 10))
    plt.plot(over_time)
    plt.title('Importance over time')
    plt.savefig(plot_folder + title + '.jpg')
    plt.close()
    return

In [ ]:
def analyze_model(model_id, name="", subdir="1", perc=None, ig=False):
    # read model
    exp_dict = utils.load_experiment_data(mlrun_id=model_id, default_id=subdir)
    feature_names = exp_dict["feature_names"]
    target_names = exp_dict["target_names"]
    data = exp_dict["data"]
    target = exp_dict["target"]
    pred = exp_dict["pred"]
    args = exp_dict["args"]
    model = exp_dict["model"]
    # create plots
    if len(target_names) == 1:
        # icp preds
        # get attrs
        attrs = get_all_attrs(model, absolute=False, perc=perc)
        create_plots(attrs, f'Saliencies/ICP_raw_{name}', feature_names, absolute=False)
        
        #attrs_block = get_attrs(model, absolute=False, block_size=12)
        #create_plots(attrs_block, f'Saliencies/Block_ICP_raw_{name}', feature_names, absolute=False)
        
        attrs_abs = get_all_attrs(model, absolute=True, perc=perc)
        create_plots(attrs_abs, f'Saliencies/ICP_abs_{name}', feature_names, absolute=True)
    else:
        # phase preds
        for idx, target_name in enumerate(target_names):
            print("Saliencies for", target_name, "with idx", idx)
            # get attrs
            #target_val = 1
            #save_name = f'Saliencies/{target_name}_target_{target_val}_raw_{name}'
            #print(save_name)
            #attrs_int = get_attrs(model, block_size=bs, target_idx=idx, desired_target_val=target_val)
            #attrs_int = get_all_attrs(model, target_idx=idx, desired_target_val=target_val, perc=perc)
            #create_plots(attrs_int, save_name, feature_names)
            
            #target_val = 0
            #save_name = f'Saliencies/{target_name}_target_{target_val}_raw_{name}'
            #print(save_name)
            #attrs_int = get_all_attrs(model, target_idx=idx, desired_target_val=target_val, perc=perc)
            #create_plots(attrs_int, save_name, feature_names)
            
            save_name = f'Saliencies/{target_name}_target_all_raw_{name}'
            print(save_name)
            # get attrs
            attrs = get_all_attrs(model, target_idx=idx, perc=perc, agg=False, ig=ig)
            # save attrs
            #print(len(attrs), attrs[0].shape, attrs[1].shape)
            #summed_per_pat = [attr.sum(axis=0) for attr in attrs]
            #print("summed: len: ", len(summed_per_pat), summed_per_pat[0].shape, summed_per_pat[0][0])
            #print("type: ", type(summed_per_pat))
            
            # plot attrs
            attrs = utils.pad_and_mask(attrs, attr=True)
            print("Final attrs shape: ", attrs.shape)
            create_plots(attrs, save_name, feature_names)
            #attrs_abs = get_attrs(model, absolute=True, block_size=bs, idx=idx)
            #create_plots(attrs_abs, f'Saliencies/{target_name}_abs_{name}_{bs}', feature_names, absolute=True)
            

In [ ]:
def get_sal_list():
    attrs = get_all_attrs(model, target_idx=idx, perc=perc, agg=False)
    summed_per_pat = [attr.sum(axis=0) for attr in attrs]
    return summed_per_pat

In [ ]:
phase_id = 'e075eaaf10fe48f2aa3d738cc4495fd7'
icp_id = 'f0d90ecf970f467fa23204e6f4490bd3'

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#analyze_model(icp_id, "ICP", perc=0.1)
#analyze_model(phase_id, "phase", perc=1.0, ig=0)

In [ ]:
def get_sal_list(model, idx, perc, agg, ds=None, ig=False, **sal_kwargs):
    model.train()
    attrs = get_all_attrs(model, target_idx=idx, perc=perc, agg=False, ds=ds, ig=ig, **sal_kwargs)
    summed_per_pat = [attr for attr in attrs]
    return summed_per_pat


In [ ]:
import pandas as pd

def get_rank(sal_list, feature_names):
    normed_sal_list = [pat / np.abs(pat).sum() for pat in sal_list]
    feat_sal_list = [pat.sum(0) for pat in normed_sal_list]
    feat_sal = np.sum(feat_sal_list, axis=0)
    feat_sal = feat_sal / np.abs(feat_sal).max()

    rank_df = pd.DataFrame({"val": feat_sal, "name": feature_names, "val_abs": np.abs(feat_sal)})
    rank_df = rank_df.sort_values(by="val_abs", ascending=False)

    rank_names = rank_df["name"].tolist()
    rank_idcs = [list(feature_names).index(name) for name in rank_names]
    rank_scores = rank_df["val"]
    return rank_names, rank_idcs, rank_scores

In [ ]:
from sklearn.metrics import average_precision_score

def eval_model(model, noise_std=1.0, noise_idcs=None, ds=None, dl=None, batch_size=512, median=None):
    if ds is None:
        ds = model.val_dataloader().dataset
        
    if dl is None:
        dl = torch.utils.data.DataLoader(ds, batch_size=batch_size)
    
    val_preds = []
    val_targets = []
    for pat_data, pat_target, idx, lens in dl:
        bs = pat_data.shape[0]
        pat_data = pat_data.cuda(non_blocking=True)
        # potentially set some feature to median
        if median is not None:
            pat_data[:, :, noise_idcs] = median[0, noise_idcs].to("cuda")
        # potentially add noise
        if noise_idcs is not None and noise_std != 0:
            noise_tensor = torch.zeros(bs, pat_data.shape[1], len(noise_idcs), device="cuda")
            noise_tensor = noise_tensor.normal_(std=noise_std)
            pat_data[:, :, noise_idcs] += noise_tensor
        # pred
        with torch.no_grad():
            pat_pred = model(pat_data)
        # select phase
        pat_target = pat_target[:, :, phase_idx].cpu()
        pat_pred = pat_pred[:, :, phase_idx].cpu()
        # cut lens
        for idx, len_ in enumerate(lens):
            cut_pred = pat_pred[idx, :len_].flatten()
            cut_target = pat_target[idx, :len_].flatten()
            val_preds.append(cut_pred)
            val_targets.append(cut_target)
    # flatten list to array
    val_preds = np.array([pred for pat in val_preds for pred in pat])
    val_targets = np.array([target for pat in val_targets for target in pat])
    # remove nans
    nan_mask = np.isnan(val_targets)
    val_targets = val_targets[~nan_mask]
    val_preds = val_preds[~nan_mask]
    # calc score
    score = average_precision_score(val_targets, val_preds)
    return score

In [ ]:
import matplotlib.pyplot as plt

def calc_remove_feat_score(model, used_idcs, noise_lvls=None, median=None, plot=True, ds=None, dl=None):
    # calc scores
    if median is None:
        scores = []
        for std in noise_lvls:
            score = eval_model(model, noise_idcs=used_idcs, noise_std=std, ds=ds, dl=dl)
            scores.append(score)
    else:
        score = eval_model(model, noise_idcs=used_idcs, median=median, ds=ds, dl=dl, noise_std=0.0)
        scores = [score]
    # plot
    if plot:
        plt.plot(noise_lvls, scores)
        plt.xlabel("Noise std")
        plt.ylabel("AP")
        plt.ylim(0.3, 0.7)
    return scores

In [ ]:
import os

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

def eval_heatmap(model, rank_idcs, remove_top_k, noise_lvls=None, median=None, ds=None, top=True, dl=None):
    all_scores = []
    for k in track(range(remove_top_k)):
        if k == 0:
            used_idcs = None
        elif top:
            used_idcs = rank_idcs[:k]
        else:
            used_idcs = rank_idcs[-k:]
        
        scores = calc_remove_feat_score(model, used_idcs, noise_lvls=noise_lvls, median=median, ds=ds, plot=False, dl=dl)
        all_scores.append(scores)
    all_scores = np.array(all_scores)
    return all_scores


def plot_heatmap(noise_lvls, all_scores, name, root_folder):
    plt.figure()
    sns.heatmap(all_scores, vmin=0.4, vmax=0.65)
    plt.ylabel("Feats removed")
    plt.xlabel("Std")
    plt.xticks(ticks=np.arange(len(noise_lvls)) + 0.5, labels=np.round(noise_lvls, 1))
    os.makedirs(root_folder, exist_ok=True)
    plt.savefig(os.path.join(root_folder, name + ".pdf"))
    plt.clf()

In [ ]:
def calc_dmg_distr(model, num_feats, noise_lvls=None, median=None, ds=None, dl=None):
    max_score = eval_model(model, noise_idcs=None, ds=ds, dl=dl)
    dmg_scores = []
    for feat_idx in tqdm(range(num_feats)):
        used_idcs = [feat_idx]
        feat_scores = calc_remove_feat_score(model, used_idcs, noise_lvls=noise_lvls, median=median, ds=ds, plot=False, dl=dl)
        feat_dmg_scores = np.clip(feat_scores / max_score, 0, 1)
        feat_dmg_score = np.mean(feat_dmg_scores)
        dmg_scores.append(feat_dmg_score)
    dmg_scores = np.array(dmg_scores)
    
    dmg_scores_inv = 1 / dmg_scores
    dmg_scores_norm = (dmg_scores_inv - dmg_scores_inv.min()) / (dmg_scores_inv.max() - dmg_scores_inv.min())
    dmg_distr = dmg_scores_norm / dmg_scores_norm.sum()
    
    return dmg_distr


def eval_dmg_distr(dmg_distr, rank_distr, v=0):
    assert np.isclose(dmg_distr.sum(), 1), f'{dmg_distr.sum()}'
    assert np.isclose(rank_distr.sum(), 1), f'{rank_distr.sum()}'
    # Calculate divergence between both distributions
    import scipy
    w_d = scipy.stats.wasserstein_distance(rank_distr, dmg_distr)
    kl_d = np.mean(scipy.special.kl_div(dmg_distr, rank_distr))
    mean_abs_d = np.mean(np.abs(rank_distr - dmg_distr))
    mean_sq_d = np.mean((rank_distr - dmg_distr) ** 2)
    if v:
        print("Mean sq diff: ", mean_sq_d)
        print("Wasserstein distance: ", w_d)
        print("KL divergence: ", kl_d)
        print("Mean abs diff: ", mean_abs_d)
    return w_d, kl_d, mean_abs_d, mean_sq_d
        

In [ ]:
def get_sal_name(ig, rand, noise_tunnel, sg_std, zero_baseline, median_baseline, median_step_baseline, num_baselines):
    name = f'{"ig" if ig else ("sg" if not rand else "rand")}'
    if noise_tunnel and not rand:
        name += "_nt"
        name += f'_{sg_std}'
    if ig:
        if zero_baseline:
            name += "_zero"
        elif median_baseline:
            name += "_med"
        elif median_step_baseline:
            name += "_medStep"
        elif num_baselines > 1:
            name += "_" + str(num_baselines)
    return name

In [ ]:
def get_sal_all(model, feature_names, ig=False, rand=False, median_baseline=False, 
                median_step_baseline=False, zero_baseline=False, num_baselines=1, sg_std=0.01, noise_tunnel=True,
                ):
    name = get_sal_name(ig, rand, noise_tunnel, sg_std, zero_baseline, median_baseline, median_step_baseline, num_baselines)
    
    if rand:
        rank_idcs = np.arange(len(feature_names))
        np.random.shuffle(rank_idcs)
        rank_names = feature_names
        
        rank_scores = np.ones(len(feature_names))
        #rank_scores = np.linspace(1, 0.001, len(feature_names))
        rank_scores /= rank_scores.sum()
        sal_list = None
    else:
        sal_list = get_sal_list(model, 0, phase_idx, False, ig=ig, median_baseline=median_baseline,
                                median_step_baseline=median_step_baseline, num_baselines=num_baselines,
                                sg_std=sg_std, zero_baseline=zero_baseline, noise_tunnel=noise_tunnel)
        rank_names, rank_idcs, rank_scores = get_rank(sal_list, feature_names)
    return name, rank_idcs, rank_names, rank_scores, sal_list

In [ ]:
def eval_sal_distr(dmg_distr, rank_idcs, rank_scores, name, root_folder):
    # calc dmg distribution and calculate divergence to rank_scores
    # dmg_distr = calc_dmg_distr(model, rank_idcs, rank_scores, noise_lvls, ds=ds, dl=dl)
    model_dmg_distr = dmg_distr[rank_idcs] # Rearrange order according to ranking
    rank_scores = rank_scores / np.abs(rank_scores).sum()
    # save sal distr
    distr_folder = os.path.join(root_folder, "distrs")
    os.makedirs(distr_folder, exist_ok=True)
    np.save(os.path.join(distr_folder, name + "_imp.npy"), rank_scores)
    # eval distr
    rank_distr = np.abs(rank_scores)

    eval_dmg_distr(model_dmg_distr, rank_distr)

    return rank_distr, model_dmg_distr

In [ ]:
def eval_sal_method(model, feature_names, dmg_distr=None, noise_lvls=None, median=None, plot=False, ds=None, dl=None, **sal_kwargs):
    name, rank_idcs, rank_names, rank_scores, sal_list = get_sal_all(model, feature_names, **sal_kwargs)
    if ds is None:
        ds = model.val_dataloader().dataset
    if dl is None:
        dl = torch.utils.data.DataLoader(ds, batch_size=256, pin_memory=True)
    root_folder = "sal_eval"
    
    if dmg_distr is not None:
        dmg_distr = np.copy(dmg_distr)
        rank_distr, dmg_distr = eval_sal_distr(dmg_distr, rank_idcs, rank_scores, name, root_folder)
        return rank_distr, dmg_distr, name
    else:
        remove_top_k = 20
        # remove top and bottom k features
        all_scores_top = eval_heatmap(model, rank_idcs, remove_top_k, noise_lvls=noise_lvls, median=median, ds=ds, top=True, dl=dl)
        all_scores_bot = eval_heatmap(model, rank_idcs, remove_top_k, noise_lvls=noise_lvls, median=median, ds=ds, top=False, dl=dl)
        # save arrays:
        eval_array_folder = os.path.join(root_folder, "eval_arrays")
        os.makedirs(eval_array_folder, exist_ok=True)
        np.save(os.path.join(eval_array_folder, name + "_top.npy"), all_scores_top)
        np.save(os.path.join(eval_array_folder, name + "_bot.npy"), all_scores_bot)
        # save saliency list:
        if sal_list is not None:
            sal_folder = os.path.join(root_folder, "saliencies")
            os.makedirs(sal_folder, exist_ok=True)
            np.save(os.path.join(sal_folder, name + ".npy"), sal_list)
        # save plots
        if plot:
            heatmap_folder = os.path.join(root_folder, "heatmaps")
            os.makedirs(heatmap_folder, exist_ok=True)
            plot_heatmap(noise_lvls, all_scores_top, name + "_top", heatmap_folder)
            plot_heatmap(noise_lvls, all_scores_bot, name + "_bot", heatmap_folder)
    
        return all_scores_top, all_scores_bot, name

In [ ]:
torch.backends.cudnn.benchmark = True
exp_dict = utils.load_experiment_data(mlrun_id=phase_id, default_id="1")
feature_names = exp_dict["feature_names"]
model = exp_dict["model"]
model.cuda()
model.eval()

phase = "long"
phase_idx = 1 if phase == "long" else 0

In [ ]:
def get_dmg_distr(model, noise_lvls=None, median=None):
    ds = model.val_dataloader().dataset
    dl = torch.utils.data.DataLoader(ds, batch_size=256, pin_memory=True)

    num_features = len(feature_names)
    dmg_distr = calc_dmg_distr(model, num_features, noise_lvls=noise_lvls, median=median, ds=ds, dl=dl)
    return dmg_distr

In [ ]:
def plot_distrs(rank_distr, dmg_distr):
    x_axis = range(len(rank_distr))
    plt.bar(x_axis, rank_distr, label="Importance")
    plt.bar(x_axis, dmg_distr, label="Damage", alpha=0.6)
    plt.legend()
    plt.ylim(0, max(dmg_distr + 0.01))

In [ ]:
np.save(os.path.join('sal_eval', "feat_names.npy"), feature_names)

In [ ]:
eval_args = (model, feature_names)

In [ ]:
eval_kwarg_list=[{'rand': True},

                 {'sg_std': 0.01},
                 {'sg_std': 0.02},
                 {'sg_std': 0.005},
                 {'sg_std': 0.1},
                 {'sg_std': 1.0},


                 {'ig': True, 'noise_tunnel': False, 'median_baseline': True},
                 {'ig': True, 'noise_tunnel': True, 'median_baseline': True},
                 {'ig': True, 'noise_tunnel': False, 'median_step_baseline': True},
                 #{'ig': True, 'noise_tunnel': True, 'median_step_baseline': True},       
                 #{'ig': True, 'noise_tunnel': True, 'num_baselines': 5},     
                 #{'ig': True, 'noise_tunnel': False, 'num_baselines': 5},        
                 #{'ig': True, 'noise_tunnel': True, 'zero_baseline': True},     
                 {'ig': True, 'noise_tunnel': False, 'zero_baseline': True},   
                ]

In [ ]:
eval_kwarg_list_debug =[{'rand': True},

                 {'sg_std': 0.01},
                
                ]

In [ ]:
def minmax(col):
    return (col - col.min()) / (col.max() - col.min())

In [ ]:
def minmax_sort_df(df):
    df.index = df["name"]
    df = df.drop(columns=["name"])
    
    minmax_names = []
    for col in df.columns:
        minmax_name = col + "_minmax"
        df[minmax_name] = np.round(minmax(df[col]), 3)
        minmax_names.append(minmax_name)
    df["Agg"] = df[minmax_names].mean(axis=1)
    df = df.sort_values(by="Agg", ascending=True)
    return df

In [ ]:
def store_dmg_df(model, dmg_distr, apply_noise, eval_args, eval_kwarg_list):
    df = pd.DataFrame()
    for eval_kwargs in eval_kwarg_list:
        eval_kwargs['dmg_distr'] = dmg_distr
        rank_distr, model_dmg_distr, name = eval_sal_method(*eval_args, **eval_kwargs)
        del eval_kwargs['dmg_distr']
        w_d, kl_d, abs_d, sq_d = eval_dmg_distr(model_dmg_distr, rank_distr, v=0)
        df_dict_row = {"name": [name], 'Wasserstein': [w_d], 'KL': [kl_d], 'Mean Squared': [sq_d]}
        df = df.append(pd.DataFrame(df_dict_row))
    df = minmax_sort_df(df)
    df.to_csv(f"sal_eval/dmg_eval_{'noise' if apply_noise else 'med'}.csv")
    return df

In [ ]:
def store_rank_df(model, noise_lvls, median_train, apply_noise, eval_args, eval_kwarg_list):
    df = pd.DataFrame()
    for eval_kwargs in eval_kwarg_list:
        eval_kwargs['noise_lvls'] = noise_lvls
        eval_kwargs['median'] = median_train
        scores_top, scores_bot, name = eval_sal_method(*eval_args, **eval_kwargs)
        del eval_kwargs['noise_lvls']
        del eval_kwargs['median']

        mean_top, mean_bot = np.mean(scores_top), np.mean(scores_bot)
        df_dict_row = {"name": [name], 'Top': [mean_top], 'Bot': [mean_bot]}
        df = df.append(pd.DataFrame(df_dict_row))
    df["Bot"] = 1 - df["Bot"]  
    df = minmax_sort_df(df)
    
    df.to_csv(f"sal_eval/rank_eval_{'noise' if apply_noise else 'med'}.csv")
    return df

In [ ]:
def get_settings(model, apply_noise):
    noise_lvls = None
    median_train = None
    
    distr_folder = os.path.join("sal_eval", "distrs")
    os.makedirs(distr_folder, exist_ok=True)
    
    if apply_noise:
        noise_lvls = np.arange(0.05, 2, 0.05)
        dmg_distr = get_dmg_distr(model, noise_lvls)
        np.save(os.path.join(distr_folder, "dmg_distr_std.npy"), dmg_distr)
    else:
        median_train = calc_median(model.train_dataloader().dataset)
        dmg_distr = get_dmg_distr(model, median=median_train)
        np.save(os.path.join(distr_folder, "dmg_distr_med.npy"), dmg_distr)
    plt.bar(range(len(dmg_distr_med)), sorted(dmg_distr, reverse=True))
    return noise_lvls, median_train, dmg_distr

In [ ]:
def compare_sals(model, eval_args, eval_kwarg_list, apply_noise=False, settings=None):
    if settings is None:
        settings = get_settings(model, apply_noise)
    noise_lvls, median_train, dmg_distr = settings
    
    # start first evaluation based on single feats
    df_dmg = store_dmg_df(model, dmg_distr, apply_noise, eval_args, eval_kwarg_list)
    print(df_dmg)
    # start second rank-based evaluation:
    df_rank = store_rank_df(model, noise_lvls, median_train, apply_noise, eval_args, eval_kwarg_list)
    print(df_rank)
    
    return df_dmg, df_rank

In [ ]:
settings = get_settings(model, False)

In [ ]:
compare_sals(model, eval_args, eval_kwarg_list, apply_noise=False, settings=settings)

In [ ]:
settings = get_settings(model, True)

In [ ]:
compare_sals(model, eval_args, eval_kwarg_list, apply_noise=True)

In [ ]:
noise_lvls

In [ ]:
eval_kwargs = {'dmg_distr': dmg_distr,
               'rand':True}
rank_distr, model_dmg_distr = eval_sal_method(*eval_args, **eval_kwargs)
plot_distrs(rank_distr, model_dmg_distr)

In [ ]:
eval_kwargs = {'dmg_distr': dmg_distr}
rank_distr, model_dmg_distr = eval_sal_method(*eval_args, **eval_kwargs)
plot_distrs(rank_distr, model_dmg_distr)

In [ ]:
eval_kwargs = {'dmg_distr': dmg_distr,
               'ig': True}
rank_distr, model_dmg_distr = eval_sal_method(*eval_args, **eval_kwargs)
plot_distrs(rank_distr, model_dmg_distr)

In [ ]:
eval_kwargs = {'dmg_distr': dmg_distr,
               'ig': True,
               'zero_baseline': True}
rank_distr, model_dmg_distr = eval_sal_method(*eval_args, **eval_kwargs)
plot_distrs(rank_distr, model_dmg_distr)

In [ ]:
eval_kwargs = {'dmg_distr': dmg_distr,
               'ig': True,
               'median_baseline': True}
rank_distr, model_dmg_distr = eval_sal_method(*eval_args, **eval_kwargs)
plot_distrs(rank_distr, model_dmg_distr)

In [ ]:
# Random attribution

In [ ]:
noise_lvls = [0.2, 0.5]

In [ ]:
scores_top_rand, scores_bot_rand, name = eval_sal_method(model, feature_names, rand=True, noise_lvls=noise_lvls)
print(scores_top_rand.mean(), scores_bot_rand.mean())

In [ ]:
# Smoothgrad used so far:

In [ ]:
scores_top_smooth, scores_bot_smooth = eval_sal_method(model, feature_names, rand=False, ig=False)
print(scores_top_smooth.mean(), scores_bot_smooth.mean())

In [ ]:
# Integrated Gradients:

In [ ]:
scores_top_ig, scores_bot_ig = eval_sal_method(model, feature_names, rand=False, ig=True)
print(scores_top_ig.mean(), scores_bot_ig.mean())

In [ ]:
scores_top_ig_med, scores_bot_ig_med = eval_sal_method(model, feature_names, rand=False, ig=True, median_baseline=True)
print(scores_top_ig_med.mean(), scores_bot_ig_med.mean())

In [ ]:
scores_top_ig_medstep, scores_bot_ig_medstep = eval_sal_method(model, feature_names, rand=False, ig=True, median_step_baseline=True)
print(scores_top_ig_medstep.mean(), scores_bot_ig_medstep.mean())

In [ ]:
scores_zero_bl_top, scores_zero_bl_bot = eval_sal_method(model, feature_names, rand=False, ig=True, zero_baseline=True)
print(scores_zero_bl_top.mean(), scores_zero_bl_bot.mean())

In [ ]:
scores_top_3, scores_bot_3 = eval_sal_method(model, feature_names, rand=False, ig=True, num_baselines=3)
print(scores_top_3.mean(), scores_bot_3.mean())

In [ ]:
scores_top_5, scores_bot_5 = eval_sal_method(model, feature_names, rand=False, ig=True, num_baselines=5)
print(scores_top_5.mean(), scores_bot_5.mean())

In [ ]:
# IG no smoothgrad

In [ ]:
scores_top_ig_med, scores_bot_ig_med = eval_sal_method(model, feature_names, rand=False, ig=True, median_baseline=True, noise_tunnel=False)
print(scores_top_ig_med.mean(), scores_bot_ig_med.mean())

In [ ]:
scores_top_ig_med, scores_bot_ig_med = eval_sal_method(model, feature_names, rand=False, ig=True, zero_baseline=True, noise_tunnel=False)
print(scores_top_ig_med.mean(), scores_bot_ig_med.mean())

In [ ]:
# Pure Smoothgrad

In [ ]:
scores_smooth_top, scores_smooth_bot = eval_sal_method(model, feature_names, sg_std=0.1)
print(scores_smooth_top.mean(), scores_smooth_bot.mean())

In [ ]:
scores_smooth_top, scores_smooth_bot = eval_sal_method(model, feature_names, sg_std=0.05)
print(scores_smooth_top.mean(), scores_smooth_bot.mean())

In [ ]:
scores_smooth_top, scores_smooth_bot = eval_sal_method(model, feature_names, sg_std=0.002)
print(scores_smooth_top.mean(), scores_smooth_bot.mean())

In [ ]:
scores_smooth_top, scores_smooth_bot = eval_sal_method(model, feature_names, sg_std=0.0001)
print(scores_smooth_top.mean(), scores_smooth_bot.mean())